In [1]:
import pandas as pd
import requests
import json
import numpy as np
from itertools import product

In [2]:
api = 'https://api.sleeper.com'

league_id = '916736605384298496'

In [3]:
league = f'{api}/v1/league/{league_id}'

In [4]:
teams = f'{league}/users'
lf = pd.json_normalize(json.loads(requests.get(teams).text))
lf = lf[['user_id','display_name']]

In [5]:
rosters = f'{league}/rosters'
rf = pd.json_normalize(json.loads(requests.get(rosters).text))
rf = rf[['owner_id','roster_id','settings.wins', 'settings.fpts','settings.fpts_decimal']]

In [6]:
df = lf.merge(rf, left_on='user_id', right_on = 'owner_id', how='left')
df['pts'] = (df['settings.fpts'].astype(str)+'.'+df['settings.fpts_decimal'].astype(str)).astype(float)
df.drop(['settings.fpts','settings.fpts_decimal'], axis = 1, inplace=True)
df.columns = ['user_id', 'display_name', 'owner_id','roster_id', 'wins', 'pts_for']

In [7]:
df = df.sort_values(by=['wins','pts_for'], ascending=[False,False]).reset_index(drop=True)
df

,user_id,display_name,owner_id,roster_id,wins,pts_for
0,725887513603932160,lakedawson,725887513603932160,4,13,2075.80
1,792485519014625280,jprice08,792485519014625280,8,9,1962.14
2,340506090141708288,jspardue,340506090141708288,9,9,1889.34
3,792484068049686528,yejinkim4,792484068049686528,6,8,1584.76
4,342852494541471744,Pangle,342852494541471744,12,7,1829.14
5,340558818712760320,ndkauffm,340558818712760320,3,7,1730.70
6,335516722092052480,itsmo,335516722092052480,1,7,1726.80
7,335446335220121600,windleypratt,335446335220121600,2,6,1962.14
8,725868672496762880,TonyLucadamo,725868672496762880,10,5,1800.88
9,340205118542610432,rtpleasa,340205118542610432,11,5,1615.70


In [8]:
df.to_csv('rosters.csv',index=False)

In [28]:
week = 14
matchups = f'{league}/matchups/{str(week)}'
mf = pd.json_normalize(json.loads(requests.get(matchups).text))
mf = mf[['matchup_id','roster_id']]
mf = mf.merge(mf, on ='matchup_id')
mf.columns = ['matchup','roster_id','opponent']
mf = mf[mf['roster_id'] != mf['opponent']].reset_index(drop=True)



weeks = range(week-1)
for wk in weeks:
    wk += 1
    m = f'{league}/matchups/{str(wk)}'
    ff = pd.json_normalize(json.loads(requests.get(m).text))
    ff = ff[['roster_id','points']]
    ff.columns = ['roster_id',f'wk{wk}']
    if wk == 1:
        wf = ff.copy()
    else:
        wf = wf.merge(ff, on='roster_id',how='left')

# Function to remove outliers using IQR
def remove_outliers(df):
    q1 = df.quantile(0.20)
    q3 = df.quantile(0.80)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return df[(df > lower_bound) & (df < upper_bound)]

# Remove outliers for each team and then calculate avg and std
filtered_scores = wf.set_index('roster_id').apply(remove_outliers, axis=1)
a = filtered_scores.mean(axis=1).reset_index()
a.columns = ['roster_id', 'avg']
b = filtered_scores.std(axis=1).reset_index()
b.columns = ['roster_id', 'std']


#a = wf.set_index('roster_id', drop=True).mean(axis=1).reset_index()
#a.columns = ['roster_id','avg']
#b = wf.set_index('roster_id', drop=True).std(axis=1).reset_index()
#b.columns = ['roster_id','std']

wf = a.merge(b)
mf = mf.merge(wf)

In [29]:
mf

,matchup,roster_id,opponent,avg,std
0,6,1,4,129.306667,13.359922
1,6,4,1,159.676923,28.867995
2,3,2,12,150.933846,26.854731
3,3,12,2,140.703077,23.470948
4,2,3,6,125.690000,20.082246
5,2,6,3,121.904615,21.783182
6,5,5,10,103.867692,19.935611
7,5,10,5,138.529231,26.552134
8,1,7,8,110.395385,24.390265
9,1,8,7,150.933846,28.652689


In [11]:
df = df.merge(mf, on = 'roster_id', how='left')
df = df.merge(df[['roster_id','display_name']], left_on='opponent',right_on='roster_id', how='left')
df.columns = ['user_id', 'username', 'owner_id', 'roster_id', 'wins','pts_for', 'matchup', 'opponent_id','avg','std','roster_id_y', 'opponent']
df= df[['username', 'wins','pts_for','avg','std', 'matchup','opponent']]

In [12]:
df.to_csv('wk14_matchups.csv',index=False)

In [23]:
# Your DataFrame 'df' should already be loaded in your Jupyter environment

# Function to simulate a single matchup and update the dataframe
def simulate_matchup(df, winner, loser):
    df.loc[df['username'] == winner, 'wins'] += 1
    df.loc[df['username'] == winner, 'won'] = True
    df.loc[df['username'] == loser, 'won'] = False

# Function to calculate the playoffs based on the current standings
def calculate_playoffs(df):
    df_sorted = df.sort_values(by=['wins', 'pts_for'], ascending=[False, False])
    df_sorted['playoffs'] = False
    df_sorted.iloc[:6, df_sorted.columns.get_loc('playoffs')] = True
    return df_sorted

# Generate all possible win/loss combinations for the matchups
matchup_combinations = list(product([True, False], repeat=6))

# Initialize a list to store all scenario dataframes
scenario_dfs = []

# Simulate each scenario
for combination in matchup_combinations:
    temp_df = df.copy()

    # Generate random scores based on avg and std, update the pts_for
    for team in temp_df['username']:
        avg_score = temp_df.loc[temp_df['username'] == team, 'avg'].iloc[0]
        std_dev = temp_df.loc[temp_df['username'] == team, 'std'].iloc[0]
        random_score = np.random.normal(avg_score, std_dev)
        temp_df.loc[temp_df['username'] == team, 'pts_for'] = random_score

    # Simulate each matchup
    for i, result in enumerate(combination):
        matchup_id = i + 1
        teams = temp_df[temp_df['matchup'] == matchup_id]
        winner, loser = (teams.iloc[0]['username'], teams.iloc[1]['username']) if result else (teams.iloc[1]['username'], teams.iloc[0]['username'])
        simulate_matchup(temp_df, winner, loser)
    
    # Calculate playoffs and add to the scenario list
    ranked_df = calculate_playoffs(temp_df)
    scenario_dfs.append(ranked_df)

# Initialize a dictionary to count playoffs appearances for each team
playoff_counts = {team: 0 for team in df['username']}

# After generating all scenarios, count playoffs appearances
for scenario_df in scenario_dfs:
    teams_in_playoffs = scenario_df[scenario_df['playoffs']]['username']
    for team in teams_in_playoffs:
        playoff_counts[team] += 1

# Calculate the percentage of playoffs appearances
total_scenarios = len(scenario_dfs)
playoff_percentages = {team: (count / total_scenarios) * 100 for team, count in playoff_counts.items()}

# Display the playoff percentages
playoff_percentages

{'lakedawson': 100.0,
 'jprice08': 100.0,
 'jspardue': 100.0,
 'yejinkim4': 95.3125,
 'Pangle': 62.5,
 'ndkauffm': 67.1875,
 'itsmo': 56.25,
 'windleypratt': 18.75,
 'TonyLucadamo': 0.0,
 'rtpleasa': 0.0,
 'abnw24': 0.0,
 'Mbalexan': 0.0}

In [18]:

# Initialize a list to store the indexes of scenarios where both teams make the playoffs
scenarios_with_both_teams_in_playoffs = []

# Iterate through each simulated outcome
for index, outcome_df in enumerate(scenario_dfs):
    # Check if both 'Pangle' and 'windleypratt' are in the playoffs
    if outcome_df[outcome_df['username'] == 'windleypratt']['playoffs'].iloc[0]:# and \
        #outcome_df[outcome_df['username'] == 'windleypratt']['playoffs'].iloc[0]:
        # Add the index of this scenario to the list
        scenarios_with_both_teams_in_playoffs.append(index)

# Display the results
print("Scenarios where both Pangle and windleypratt make the playoffs:", scenarios_with_both_teams_in_playoffs)

NameError: name 'scenario_dfs' is not defined

There are 16 scenarios where both Pangle and windleypratt make the playoffs: `[8, 10, 12, 14, 40, 42, 44, 46]`

Similarly, there are 16 scenarios where niether of us make the playoffs: `[25, 27, 29, 31, 57, 59, 61, 63]`



In [ ]:
scenes = [8, 10, 12, 14, 40, 42, 44, 46]

for o in scenes:
    print(scenario_dfs[o])

In [ ]:
# Initialize a dictionary to count playoffs appearances for each team
playoff_counts = {team: 0 for team in df['username']}

# Iterate through each scenario to count playoffs appearances
for scenario_df in scenario_dfs:
    teams_in_playoffs = scenario_df[scenario_df['playoffs']]['username']
    for team in teams_in_playoffs:
        playoff_counts[team] += 1

# Calculate the percentage of playoffs appearances
total_scenarios = len(scenario_dfs)
playoff_percentages = {team: (count / total_scenarios) * 100 for team, count in playoff_counts.items()}

# Display the playoff percentages
playoff_percentages

In [ ]:
week = 14
weeks = range(week-1)
for wk in weeks:
    wk += 1
    m = f'{league}/matchups/{str(wk)}'
    ff = pd.json_normalize(json.loads(requests.get(m).text))
    ff = ff[['roster_id','points']]
    ff.columns = ['roster_id',f'wk{wk}']
    if wk == 1:
        wf = ff.copy()
    else:
        wf = wf.merge(ff, on='roster_id',how='left')
        
a = wf.set_index('roster_id', drop=True).mean(axis=1).reset_index()
a.columns = ['roster_id','avg']
b = wf.set_index('roster_id', drop=True).std(axis=1).reset_index()
b.columns = ['roster_id','std']

wf = a.merge(b)
wf